## Kibana Installation

### Installing Kibana

##### Kibana is an open source data visualization plugin for Elasticsearch. It provides visualization capabilities on top of the content indexed on an Elasticsearch cluster. Users can create bar, line and scatter plots, or pie charts and maps on top of large volumes of data. 

##### According to the official documentation, we must install Kibana only after installing Elasticsearch to ensure that the components depending on each product are in place.

In [ ]:
%%bash
sudo apt install -y kibana

##### Now we will change the permissions of a specific directory to prevent kibana from collisionin with x-pack.

In [ ]:
%%bash
sudo chmod =rwx,g+s /usr/share/kibana/optimize/bundles

### Edit Kibana's configuration file

We would like to edit some of teh default configuration set in Kibana's configuration file to define our host's IP address which will be used for presenting the web interface on port 5601. The default IP is set to the one associated to localhost. Leaving this default seeting will not work. In addition, we need to provide kibana with the kibana's user credentials set previously.

In [ ]:
%%bash
temp_ens3=$(ip addr show ens3 | grep -Po 'inet \K[\d.]+')
sudo sed -i "2i$temp_ens3 $HOSTNAME" /etc/hosts 
sudo sed -i "8iserver.host: $temp_ens3" /etc/kibana/kibana.yml 
sudo sed -i '49ielasticsearch.username: "kibana"' /etc/kibana/kibana.yml 
sudo sed -i '50ielasticsearch.password: "elasticsiem"' /etc/kibana/kibana.yml 

### Start and enable Kibana

In [ ]:
%%bash
sudo systemctl enable kibana
sudo systemctl start kibana

##### Note: We encourage you to install additionall plugins such as Vega and Network using the commands below. Some of the ones stated in the following URL are still not available for our current Kibana version 7.2.0

https://www.elastic.co/guide/en/kibana/current/known-plugins.html

In [ ]:
#%%bash
#sudo /usr/share/kibana/bin/kibana-plugin install https://github.com/nyurik/kibana-vega-vis/releases/download/v1.2.0/vega_vis-1.2.0--for-Kibana-6.2.4.zip --allow-root
#sudo -u kibana /usr/share/kibana/bin/kibana-plugin install vega

##### By default kibana presents it's web interface on port 5601. We will use a reverse proxy to forward traffic to port 80. To achieve this, we must first install NGINS and set it as our reverse proxy.

In [ ]:
%%bash
sudo apt install -y -q nginx

### Seting up an Administrative kibana user using SSL

##### We will start by creating an "admin" account using ssl. The command presented below will require a password and store it in the "/etc/nginx/htpasswd.users" directory. Type the following command in the terminal and confirm your passowrd

~~~~
echo "admin:`openssl passwd -apr1`" | sudo tee -a /etc/nginx/htpasswd.users
~~~~

##### We can verify if the password has been saved by checking the contents of the destination file using the following command:

In [ ]:
%%bash
sudo cat /etc/nginx/htpasswd.users

### Configure NGINX

NGINX cofinguration can be a little bit tricky; due to this, we will copy a template from the files directory and set your hostname and your local IP in its configuration file. This file is created in the "etc/nginx/sites-available/" directory but nothing will happen even after setting the configuration. We need to generate a symbolic link from the "/etc/nginx/sites-available/nginx_proxy" file into the "/etc/nginx/sites-enabled/" directory. NGINX will read the files in the sites-enabled make the setup according to the files in it.

In [ ]:
%%bash
HOSTNAME=hostname -f
temp_ens3=$(ip addr show ens3 | grep -Po 'inet \K[\d.]+')

sudo rm /etc/nginx/sites-available/default /etc/nginx/sites-enabled/default 
sudo cp files/default /etc/nginx/sites-available/ 

sudo sed -i "s/<YOUR-HOSTNAME-HERE>/$HOSTNAME/g" /etc/nginx/sites-available/default
sudo sed -i "s/<YOUR-IP-HERE>/$temp_ens3/g" /etc/nginx/sites-available/default

sudo ln -s /etc/nginx/sites-available/default /etc/nginx/sites-enabled/default

sudo cat /etc/nginx/sites-available/default

### Start and Enable NGINX

In [ ]:
%%bash
sudo nginx -t
sudo systemctl start nginx
sudo systemctl reload nginx
sudo systemctl enable nginx

##### What is going on? Now we have Kibana in the following two sites?

~~~~
<YOUR-PUBLIC-IP>:<5601>
<YOUR-PUBLIC-IP>:<80>
~~~~

##### It happens that the native authentication is provided by X-Pack which is disabled by default. X-Pack security and machine learning are platinum features that expire after 30 days (trial license). Once this expires, your system will be completly exposed unless you provide authentication as we did. In addition, we need to secure assitional endpoint such as the REST API from the external network. To do so, we will install firewall in our system.

##### As an example you can check the status of your elasticsearch from your local computer using your credentials because port 9200 is open:

~~~~
curl -u elastic:elasticsiem -XGET <ELASTICSEARCH-IP-ADDRESS>:9200
~~~~

### Let set a local firewall.

##### Since there are some services wide open (and there will be more) to any actor who makes a request to them, we must setup a firewall that restricts unwanted traffic by setting some simple rules. For this purpose we will use the UFW firewall that is already installed in Ubuntu 18.04.

##### The following table presents the ports we are interested on:

### PORTS of Interest

| Port | Protocol | Application |
|------|----------|-------------|
| 22 | TCP | SSH |
| 53 | UDP | DNS |
| 80 | TCP | HTTP |
| 443 | TCP | HTTPS |
| 3128 | TCP | Python PIP |
| 5044 | TCP | Logstash Listening Port |
| 5601 | TCP | Kibana |
| 9200 | TCP | Elasticsearch REST API |
| 9300 | TCP | Elasticsearch Transport Module |
| 9600 | TCP | Logstash Monitoring API

##### The following table presents the rules we expect to set in our firewall:

### Expected Firewall Rules

| Number | To | Action | From |
|--------|----|--------|------|
|[ 1] |22/tcp                     |ALLOW IN    |Anywhere|                  
|[ 2] |Nginx Full                 |ALLOW IN    |Anywhere|                  
|[ 3] |80/tcp                     |ALLOW IN    |Anywhere|                  
|[ 4] |8888/tcp                   |ALLOW IN    |Anywhere|                  
|[ 5] |443                        |ALLOW IN    |Anywhere|                  
|[ 6] |5044                       |ALLOQ IN    |Anywhere|                  
|[ 7] |5601                       |DENY IN     |Anywhere|                  
|[ 8] |9200                       |DENY IN     |Anywhere|                  
|[ 9] |9300                       |DENY IN     |Anywhere|                  
|[10] |9600                       |DENY IN     |Anywhere|
|[11] |53                         |ALLOW IN    |10.1.1.0/24|
|[12] |3128                       |ALLOW IN    |Anywhere|

##### Let us chech the current status of our internal firewall UFW.

In [ ]:
%%bash
sudo ufw status

##### We can observe that our firewall is currently inactive.
##### By default, UFW is set to deny all incoming connections and allow all outgoing connections.

### WARNING! You might get locked out of your system if you don't define some rules before enabling your firewall.

##### Let us define some firewall rules of our interest in order to allow communication to go in and deny communication from going out through specific ports.

In [ ]:
%%bash
sudo ufw allow ssh
sudo ufw allow from 10.1.1.0/24 to any port 53
sudo ufw allow 'Nginx Full'
sudo ufw allow http/tcp
sudo ufw allow 443
sudo ufw allow 3128
sudo ufw allow 5044
sudo ufw deny 5601
sudo ufw allow 8888/tcp
sudo ufw deny 9200
sudo ufw deny 9300
sudo ufw deny 9600

### Check the recently added rules

In [ ]:
%%bash
sudo ufw status
sudo ufw show added

### Enable your firewall

##### Now that you have added new rules and validated that they have been set in our firewall, let us proceed to enable it with the following line:

In [ ]:
%%bash
sudo ufw enable

### List the current active rules in your firewall using the following:

In [ ]:
%%bash
sudo ufw status numbered

### We have now installed Elasticsearch, Kibana, and Secured our System using X-Pack Security and NGINX as a reverse proxy.